# **Convolutional Neural Network: Plants Classification**
**Artificial  Neural  Networks  and  Deep  Learning  -  a.y.  2022/2023**

*     <u>Marco Bendinelli</u>
> M.Sc. Computer Science Engineering at Politecnico di Milano
>
> E-mail: marco.bendinelli@mail.polimi.it
>
> Student ID : 10673478
>
> Codalab Nickname: "MarcoBendinelli"
>
> Codalab Group: "Zero Neurons Networks"
*     <u>Pietro Andrea Cirino</u>
> M.Sc. Mathematical Engineering at Politecnico di Milano
>
> E-mail: pietroandrea.cirino@mail.polimi.it
>
> Student ID : 10628055
>
> Codalab Nickname: "PietroCirino"
>
> Codalab Group: "Zero Neurons Networks"
*     <u>Marco Cayuela</u>
> M.Sc. Mathematical Engineering at Politecnico di Milano
>
> E-mail: marco.cayuela@mail.polimi.it
>
> Student ID : 10859184
>
> Codalab Nickname: "MarcoCayou"
>
> Codalab Group: "Zero Neurons Networks"

## Environment settings

### Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/Report/

/gdrive/My Drive/Report


### Libraries

In [3]:
import os
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers

print(tf.__version__)

2.9.2


### Random seed

In [4]:
# Random seed for reproducibility
SEED = 2710

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

### Suppress warnings

In [5]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Metadata

In [6]:
LABELS = ['Species1','Species2','Species3','Species4','Species5','Species6','Species7','Species8']
INPUT_SHAPE = (96, 96, 3)
IMAGE_SIZE = (INPUT_SHAPE[0], INPUT_SHAPE[1])
EPOCHS = 200
BATCH_SIZE = 32
LEARNING_RATE_FT = 5e-4
NUM_CLASSES = len(LABELS)

### Unzip dataset

In [7]:
# Unzip dataset
!unzip training_dataset_homework1.zip

# Directory
DATASET_DIR = 'training_data_final' 
print(os.listdir(DATASET_DIR))

Archive:  training_dataset_homework1.zip
   creating: training_data_final/Species1/
  inflating: training_data_final/Species1/00000.jpg  
  inflating: training_data_final/Species1/00001.jpg  
  inflating: training_data_final/Species1/00002.jpg  
  inflating: training_data_final/Species1/00003.jpg  
  inflating: training_data_final/Species1/00004.jpg  
  inflating: training_data_final/Species1/00005.jpg  
  inflating: training_data_final/Species1/00006.jpg  
  inflating: training_data_final/Species1/00007.jpg  
  inflating: training_data_final/Species1/00008.jpg  
  inflating: training_data_final/Species1/00009.jpg  
  inflating: training_data_final/Species1/00010.jpg  
  inflating: training_data_final/Species1/00011.jpg  
  inflating: training_data_final/Species1/00012.jpg  
  inflating: training_data_final/Species1/00013.jpg  
  inflating: training_data_final/Species1/00014.jpg  
  inflating: training_data_final/Species1/00015.jpg  
  inflating: training_data_final/Species1/00016.jpg 

### Image Generator

In [8]:
# Constructor
train_data_gen = ImageDataGenerator(
    # Data Augmentation
    height_shift_range=15,
    width_shift_range=15,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.85,1.15],
    fill_mode='wrap',
    validation_split=0.2
)

# Train generator
train_gen = train_data_gen.flow_from_directory(
    directory=DATASET_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    subset='training'
)

# Validation generator
valid_gen = train_data_gen.flow_from_directory(
    directory=DATASET_DIR,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
    subset='validation'
)

Found 2836 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


## RegNetX320 Transfer Learning

In [9]:
# Download the model
supernet = tfk.applications.regnet.RegNetX320(
    include_top=False,
    include_preprocessing=True,
    weights='imagenet'
)
supernet.summary()

422542312/422542312 [==============================] - 19s 0us/step
Model: "regnetx320"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 regnetx320_prestem_rescaling (  (None, None, None,   0          ['input_1[0][0]']                
 Rescaling)                     3)                                                                
                                                                                                  
 regnetx320_stem_conv (Conv2D)  (None, None, None,   864         ['regnetx320_prestem_rescaling[0]
                     

### Rebuild the top

In [12]:
# Rebuild the classifier
supernet.trainable = False

inputs = tfk.Input(shape=INPUT_SHAPE)

x = supernet(inputs)

x = tfkl.GlobalAveragePooling2D()(x)

outputs = tfkl.Dense(
    NUM_CLASSES, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(SEED),
    )(x)

# Connect input and output
model = tfk.Model(inputs=inputs, outputs=outputs, name='FinalModel')

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), 
              optimizer=tfk.optimizers.Adam(), 
              metrics=['accuracy'])
model.summary()

Model: "FinalModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 regnetx320 (Functional)     (None, None, None, 2520)  105452576 
                                                                 
 global_average_pooling2d_1   (None, 2520)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 8)                 20168     
                                                                 
Total params: 105,472,744
Trainable params: 20,168
Non-trainable params: 105,452,576
_________________________________________________________________


### Training and Validation

In [13]:
# Train the model
history = model.fit(
    x = train_gen,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = valid_gen,
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
).history

Epoch 1/200
89/89 [==============================] - 41s 293ms/step - loss: 1.2129 - accuracy: 0.5677 - val_loss: 1.0027 - val_accuracy: 0.6246
Epoch 2/200
89/89 [==============================] - 19s 213ms/step - loss: 0.8081 - accuracy: 0.7211 - val_loss: 0.8131 - val_accuracy: 0.7096
Epoch 3/200
89/89 [==============================] - 19s 214ms/step - loss: 0.7139 - accuracy: 0.7599 - val_loss: 0.7580 - val_accuracy: 0.7210
Epoch 4/200
89/89 [==============================] - 20s 225ms/step - loss: 0.6452 - accuracy: 0.7853 - val_loss: 0.6855 - val_accuracy: 0.7564
Epoch 5/200
89/89 [==============================] - 19s 212ms/step - loss: 0.5981 - accuracy: 0.7916 - val_loss: 0.6902 - val_accuracy: 0.7606
Epoch 6/200
89/89 [==============================] - 19s 215ms/step - loss: 0.5622 - accuracy: 0.8114 - val_loss: 0.6317 - val_accuracy: 0.7705
Epoch 7/200
89/89 [==============================] - 19s 214ms/step - loss: 0.5442 - accuracy: 0.8173 - val_loss: 0.6253 - val_accuracy:

## RegNetX320 Fine Tuning

### Freeze the first 80 layers

In [14]:
# Set all layers to True
model.get_layer('regnetx320').trainable = True

# Freeze the first 80 layers
for i, layer in enumerate(model.get_layer('regnetx320').layers[:81]):
  layer.trainable=False

for i, layer in enumerate(model.get_layer('regnetx320').layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 regnetx320_prestem_rescaling False
2 regnetx320_stem_conv False
3 regnetx320_stem_bn False
4 regnetx320_stem_relu False
5 regnetx320_Stage_0_XBlock_0_conv_1x1_1 False
6 regnetx320_Stage_0_XBlock_0_conv_1x1_1_bn False
7 regnetx320_Stage_0_XBlock_0_conv_1x1_1_relu False
8 regnetx320_Stage_0_XBlock_0_conv_3x3 False
9 regnetx320_Stage_0_XBlock_0_conv_3x3_bn False
10 regnetx320_Stage_0_XBlock_0_conv_3x3_relu False
11 regnetx320_Stage_0_XBlock_0_conv_1x1_2 False
12 regnetx320_Stage_0_XBlock_0_skip_1x1 False
13 regnetx320_Stage_0_XBlock_0_conv_1x1_2_bn False
14 regnetx320_Stage_0_XBlock_0_skip_bn False
15 tf.__operators__.add False
16 regnetx320_Stage_0_XBlock_0_exit_relu False
17 regnetx320_Stage_0_XBlock_1_conv_1x1_1 False
18 regnetx320_Stage_0_XBlock_1_conv_1x1_1_bn False
19 regnetx320_Stage_0_XBlock_1_conv_1x1_1_relu False
20 regnetx320_Stage_0_XBlock_1_conv_3x3 False
21 regnetx320_Stage_0_XBlock_1_conv_3x3_bn False
22 regnetx320_Stage_0_XBlock_1_conv_3x3_relu False
23 r

In [15]:
# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), 
              optimizer=tfk.optimizers.Adam(LEARNING_RATE_FT), 
              metrics=['accuracy'])
model.summary()

Model: "FinalModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 regnetx320 (Functional)     (None, None, None, 2520)  105452576 
                                                                 
 global_average_pooling2d_1   (None, 2520)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 8)                 20168     
                                                                 
Total params: 105,472,744
Trainable params: 93,858,248
Non-trainable params: 11,614,496
_________________________________________________________________


### Training and Validation

In [16]:
# Train the model
history = model.fit(
    x = train_gen,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = valid_gen,
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
).history

Epoch 1/200
89/89 [==============================] - 51s 460ms/step - loss: 0.7491 - accuracy: 0.7482 - val_loss: 0.6987 - val_accuracy: 0.7790
Epoch 2/200
89/89 [==============================] - 37s 409ms/step - loss: 0.4902 - accuracy: 0.8343 - val_loss: 0.5919 - val_accuracy: 0.8088
Epoch 3/200
89/89 [==============================] - 36s 409ms/step - loss: 0.3333 - accuracy: 0.8822 - val_loss: 0.4864 - val_accuracy: 0.8513
Epoch 4/200
89/89 [==============================] - 37s 411ms/step - loss: 0.2764 - accuracy: 0.9066 - val_loss: 0.4521 - val_accuracy: 0.8584
Epoch 5/200
89/89 [==============================] - 37s 410ms/step - loss: 0.2264 - accuracy: 0.9193 - val_loss: 0.4309 - val_accuracy: 0.8697
Epoch 6/200
89/89 [==============================] - 37s 409ms/step - loss: 0.2205 - accuracy: 0.9249 - val_loss: 0.4427 - val_accuracy: 0.8711
Epoch 7/200
89/89 [==============================] - 36s 406ms/step - loss: 0.1674 - accuracy: 0.9383 - val_loss: 0.4650 - val_accuracy: